<a href="https://colab.research.google.com/github/gabriellaaileen/tensorflow-certification/blob/main/Question4_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.callbacks import EarlyStopping

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
tokenizer = Tokenizer()
data = open('/content/drive/MyDrive/poem.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

print(max_sequence_len)

16


In [21]:
label[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [42]:
#buatlah model dengan output layer total words/2 dan total word
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(64, activation='relu')),
model.add(tf.keras.layers.Dense(128, activation='relu')),
model.add(tf.keras.layers.Dense(256, activation='relu')),
model.add(tf.keras.layers.Dropout(0.2)),
model.add(tf.keras.layers.Dense(total_words, activation='softmax'))

In [41]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 15, 100)           380800    
                                                                 
 bidirectional_4 (Bidirectio  (None, 128)              84480     
 nal)                                                            
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dense_14 (Dense)            (None, 128)               8320      
                                                                 
 dense_15 (Dense)            (None, 256)               33024     
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                      

In [43]:
#Gunakan loss categorical_crossentropy
model.compile(
    loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy']
)

In [44]:
#training model 
history = model.fit(
    predictors,label,batch_size=128,epochs=400,
    validation_split=0.2
)

Epoch 1/400
102/102 [==============================] - 16s 120ms/step - loss: 7.0928 - accuracy: 0.0604 - val_loss: 7.0804 - val_accuracy: 0.0616
Epoch 2/400
102/102 [==============================] - 9s 91ms/step - loss: 6.5769 - accuracy: 0.0632 - val_loss: 7.2413 - val_accuracy: 0.0616
Epoch 3/400
102/102 [==============================] - 9s 92ms/step - loss: 6.4407 - accuracy: 0.0632 - val_loss: 7.4040 - val_accuracy: 0.0616
Epoch 4/400
102/102 [==============================] - 9s 92ms/step - loss: 6.3440 - accuracy: 0.0635 - val_loss: 7.3591 - val_accuracy: 0.0631
Epoch 5/400
102/102 [==============================] - 9s 91ms/step - loss: 6.1965 - accuracy: 0.0650 - val_loss: 7.4104 - val_accuracy: 0.0634
Epoch 6/400
102/102 [==============================] - 9s 92ms/step - loss: 6.0697 - accuracy: 0.0672 - val_loss: 7.6398 - val_accuracy: 0.0644
Epoch 7/400
102/102 [==============================] - 9s 91ms/step - loss: 5.9536 - accuracy: 0.0731 - val_loss: 7.7521 - val_accurac

In [45]:
%cd /content/drive/MyDrive/Model

/content/drive/MyDrive/Model


In [46]:
#save model
model.save('model4.h5')